# Multinomial Classification
## Softmax Regression From Scratch
This blog uses softmax regression on the iris data set to predict what type of flower each sample is based on 4 different features. I have added to this blog's code a from scratch train_test_spilt function, training and prediction functions, as well as the k-fold cross validation function.

### Get Data

In [1]:
%pylab inline
import pandas as pd
!wget -O dataset.csv https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
!head -3 dataset.csv

Populating the interactive namespace from numpy and matplotlib


--2020-09-18 19:52:03--  https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551 (4.4K) [application/x-httpd-php]
Saving to: 'dataset.csv'

     0K ....                                                  100% 8.20M=0.001s

2020-09-18 19:52:04 (8.20 MB/s) - 'dataset.csv' saved [4551/4551]



5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa


In [2]:
df = pd.read_csv('dataset.csv', names=[
  "sepal length in cm",
  "sepal width in cm",
  "petal length in cm",
  "petal width in cm",
  "class"
])

df.head()

,sepal length in cm,sepal width in cm,petal length in cm,petal width in cm,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### Prep Data

In [3]:
# Select features
X = df[["sepal length in cm",
  "sepal width in cm",
  "petal length in cm",
  "petal width in cm"
]].values.astype(np.float32)
X.shape

(150, 4)

In [4]:
# Class --> Numeric values
y = pd.factorize(df['class'])[0]
y.shape

(150,)

In [5]:
# Bias factor
X = np.hstack((np.ones((len(X), 1)), X))

m, n = X.shape
K = 3 # Classes
K, m, n

# Standardization
X[:, 1:] = (X[:, 1:] - np.mean(X[:, 1:], axis=0)) / np.std(X[:, 1:], axis=0)

np.random.seed(0)

### Model

In [6]:
def softmax(z):
    z -= np.max(z)
    return np.exp(z) / np.sum(np.exp(z))

def h(X, theta):
    return softmax(X @ theta)

def J(preds, y):
    m = preds.shape[0]
    return np.sum(- np.log(preds[np.arange(m), y]))

def T(y, K):
    """ one hot encoding """
    one_hot = np.zeros((len(y), K))
    one_hot[np.arange(len(y)), y] = 1
    return one_hot

def compute_gradient(theta, X, y):
    preds = h(X, theta)
    gradient = 1/m * X.T @ (preds - T(y, K))
    return gradient

### Split Data Up
To test properly on new data, I implemented a train_test_split function from scratch (versus using an sklearn function).

In [7]:
def my_train_test_split(X, y, train_perc):
    train_end_ind = int(X.shape[0] * train_perc)  
    train_x = X[0:train_end_ind]
    train_y = y[0:train_end_ind]
    test_x = X[train_end_ind:-1]
    test_y = y[train_end_ind:-1]

    return (train_x, train_y, test_x, test_y)


train_x, train_y, test_x, test_y = my_train_test_split(X, y, 0.8)
print("Train (x,y) shape:", train_x.shape, train_y.shape, "\nTest (x,y) shape:", test_x.shape, test_y.shape)

Train (x,y) shape: (120, 5) (120,) 
Test (x,y) shape: (29, 5) (29,)


### Training loop

In [8]:
def train_from_scratch(X, y, iters, alpha):
    theta = np.random.random((n, K))
    hist = {'loss': [], 'acc': []}
    
    for i in range(iters):
        
        gradient = compute_gradient(theta, X, y)
        theta -= alpha * gradient 

        # loss
        preds = h(X, theta) 
        loss = J(preds, y)
        hist['loss'].append(loss) 

        # acc
        c = 0 
        for j in range(len(y)): 
            if np.argmax(h(X[j], theta)) == y[j]: 
                c += 1 
        acc = c / len(y) 
        hist['acc'].append(acc) 

    return (loss, acc, theta, hist)

def test_from_scratch(X, y, theta):
    # loss
    preds = h(X, theta) 
    loss = J(preds, y)

    # acc
    c = 0 
    for j in range(len(y)): 
        if np.argmax(h(X[j], theta)) == y[j]: 
            c += 1 
    acc = c / len(y) 

    return (loss,acc)

iters = 1500 
alpha = 1e-3 

train_loss_from_scratch, train_acc_from_scratch, theta, hist = train_from_scratch(train_x, train_y, iters, alpha)
test_loss_from_scratch, test_acc_from_scratch = test_from_scratch(test_x, test_y, theta)

print(f'Training (loss, acc): {(train_loss_from_scratch,train_acc_from_scratch)}')
print(f'Testing (loss, acc): {(test_loss_from_scratch, test_acc_from_scratch)}')

Training (loss, acc): (756.2966836904976, 0.7916666666666666)
Testing (loss, acc): (153.744603026516, 0.034482758620689655)


## Logistic Regression SKlearn

In [9]:
# No fancy features we're used in the sklearn version so that we are comparing apples to apples
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split

# Init model
LR = LogisticRegression(random_state=0)
# 10-fold cv using sklearn
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
scores = cross_val_score(LR, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
kfold_sklearn_acc = mean(scores)
# Performance
print('LogisticRegression Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

LogisticRegression Mean Accuracy: 0.953 (0.043)


# Cross Validation Comparisons
We are unable to compare the from scratch and sklearn multinomial classification algorithms using the packaged pair t-test with 5x2 cross validation since it takes two sklearn models as parameters for comparison. So, we have settled to just comparing the 10-fold cross validation of each.

For the sake of learning, I have included how to use the pair t-test with 5x2 cross validation by comparing LR and LDA to show how to do that.

## K-Fold Cross Validation Comparison

In [10]:
# k_Fold Cross Validation
def cv_split(X, y, start_ind_test, end_ind_test, end_cap):
    # k-1 train segments
    train_x = np.append(X[0:start_ind_test], X[end_ind_test:end_cap], axis=0)
    train_y = np.append(y[0:start_ind_test], y[end_ind_test:end_cap], axis=0)
    # 1 test segment
    test_x = X[start_ind_test:end_ind_test]
    test_y = y[start_ind_test:end_ind_test]

    
    return (train_x, train_y, test_x, test_y)

def kfold_cv_from_scratch(train_func, test_func, X, y, iters, alpha, folds):
    hist = {'loss': [], 'acc': []} # Performance history
    fold_size = (X.shape[0]) // folds
    for i in range(folds):
        start_ind_test = i * fold_size
        end_ind_test = min(len(X) - 1, start_ind_test + fold_size)
        end_cap = fold_size * folds

        train_x, train_y, test_x, test_y = cv_split(X, y, start_ind_test, end_ind_test, end_cap)
        
        _, _, theta, _  = train_func(train_x, train_y, iters, alpha)
        loss, acc = test_func(test_x, test_y, theta)
        
        hist['loss'].append(loss)
        hist['acc'].append(acc)
    
    avg_loss, avg_acc = mean(hist['loss']), mean(hist['acc'])
    
    return (avg_loss, avg_acc)

## Multinomial Classification: Softmax From Scratch vs Multinomial Logistic sklearn
We can see that the sklearn implementation of Multinomial LR performs much better than the from scratch softmax algorithm.

In [11]:
folds = 10
_, kfold_from_scratch_acc = kfold_cv_from_scratch(train_from_scratch, test_from_scratch, X, y, iters, alpha, folds)

In [12]:
dif = kfold_from_scratch_acc - kfold_sklearn_acc
print(f'Mean Accuracy (From Scratch, SKLearn): {kfold_from_scratch_acc, kfold_sklearn_acc}, Difference: {dif}')

Mean Accuracy (From Scratch, SKLearn): (0.7185714285714285, 0.9533333333333334), Difference: -0.23476190476190484


## Paired T-Test 5x2 Cross Validation Example

In [13]:
from mlxtend.evaluate import paired_ttest_5x2cv

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
LDA = LinearDiscriminantAnalysis()

t,p = paired_ttest_5x2cv(estimator1=LR, estimator2=LDA, X=X, y=y, scoring='accuracy', random_seed=1)
# summarize
print('P-value: %.3f, t-Statistic: %.3f' % (p, t))
# interpret the result
if p <= 0.05:
    print('Difference between mean performance is probably real')
else:
    print('Algorithms probably have the same performance')

P-value: 0.122, t-Statistic: -1.861
Algorithms probably have the same performance
